# Library

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# fold path

In [2]:
raw_fold = 'D:/data/big2/BD1/raw'

In [3]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# 로드

## [로드] 종합 result

In [29]:
# about 17.0s
name = '종합_4등급 차량 리스트 양식 result 최종_수정-작업(2023.04.13_PM.02.38.43)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 38 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   차대번호        1502235 non-null  object 
 1   차량번호        1502235 non-null  object 
 2   제원관리번호      1502235 non-null  object 
 3   차종_원본       1153813 non-null  object 
 4   용도          1502235 non-null  object 
 5   최초등록일       1502235 non-null  int64  
 6   연식          1502235 non-null  int64  
 7   제작일자        1335178 non-null  float64
 8   검사유효일       1502223 non-null  float64
 9   배출가스인증번호    1382222 non-null  object 
 10  등급          1502235 non-null  int64  
 11  차명          1502235 non-null  object 
 12  차종분류        1502235 non-null  object 
 13  차종유형        1502235 non-null  object 
 14  자동차형식       1501958 non-null  object 
 15  제작사명        1492590 non-null  object 
 16  연료          1502235 non-null  object 
 17  엔진형식        1500400 non-null  object 
 18  총중량         1502234 no

In [30]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', '차종', 'Grade'],
      dtype='object')

In [31]:
df['연료'].value_counts(dropna=False)

경유             1153813
휘발유             322983
LPG(액화석유가스)      25400
기타연료                37
알코올                  1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [32]:
df['등급_수정'].value_counts(dropna=True)

4.00    1149291
3.00       4396
5.00        126
Name: 등급_수정, dtype: int64

# 전처리

## 컬럼 정리
- 차종 값 -> 차종_원본으로 이동

In [34]:
df['차종_원본'].isnull().sum()

348422

In [35]:
df[df['차종_원본'].isnull() == True].index[:5]

Int64Index([1153813, 1153814, 1153815, 1153816, 1153817], dtype='int64')

In [37]:
df.loc[[1153813, 1153814, 1153815, 1153816, 1153817], ['차종_원본', '차종']]

,차종_원본,차종
1153813,NaN,승용
1153814,NaN,승용
1153815,NaN,승용
1153816,NaN,승합
1153817,NaN,승용


In [38]:
df.loc[df['차종_원본'].isnull() == True, '차종_원본'] = df.loc[df['차종_원본'].isnull() == True, '차종']

In [39]:
df['차종_원본'].isnull().sum()

0

In [40]:
df.loc[[1153813, 1153814, 1153815, 1153816, 1153817], ['차종_원본', '차종']]

,차종_원본,차종
1153813,승용,승용
1153814,승용,승용
1153815,승용,승용
1153816,승합,승합
1153817,승용,승용


In [41]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', '차종', 'Grade'],
      dtype='object')

In [42]:
df.drop(['차종', 'Grade'], axis=1, inplace=True)
df.rename(columns={'차종_원본':'차종'}, inplace=True)
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고'],
      dtype='object')

## 기타연료 수정
- 에어스트림탐지견수송트레일러 제외 나머지 36대 '휘발유'로 수정

In [43]:
df.loc[df['연료'] == '기타연료', ['연료', '차명']]

,연료,차명
1502196,기타연료,캐피탈
1502197,기타연료,르망
1502198,기타연료,도요다크라운로얄싸롱
1502199,기타연료,캐피탈
1502200,기타연료,로얄살롱2.0AT
1502201,기타연료,스텔라오토매틱
1502202,기타연료,머큐리 세이블
1502204,기타연료,벤즈 230E
1502205,기타연료,캐피탈
1502206,기타연료,벤즈230E


In [44]:
df.loc[df['연료'] == '기타연료', ['연료', '차명']].shape

(37, 2)

In [45]:
df.loc[(df['연료'] == '기타연료') & (df['차명'] != '에어스트림탐지견수송트레일러'), ['연료', '비고']] = ['휘발유', '기타연료에서 휘발유로 변경']

In [46]:
df['연료'].value_counts(dropna=False)

경유             1153813
휘발유             323019
LPG(액화석유가스)      25400
알코올                  1
기타연료                 1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [47]:
df.loc[df['비고'] == '기타연료에서 휘발유로 변경'].shape

(36, 36)

In [48]:
df.shape

(1502235, 36)

### [출력] 기타연료 수정한 파일

In [49]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.04.13_PM.04.25.49'

In [50]:
# about 21.5s
df.to_csv(os.path.join(raw_fold, f'종합_4등급 차량 리스트 양식 result 최종_수정-작업({today_date}).csv'), index=False)

In [51]:
# about 14m 33.5s
df.iloc[:700000].to_excel(os.path.join(raw_fold, f'종합_4등급 차량 리스트 양식 result 최종_수정-작업01({today_date}).xlsx'), index=False)
df.iloc[700000:].to_excel(os.path.join(raw_fold, f'종합_4등급 차량 리스트 양식 result 최종_수정-작업02({today_date}).xlsx'), index=False)

# 코드 마지막